# Hidden Layer Quickstart

This notebook verifies your environment and demonstrates the core harness APIs.
Feel free to edit and save under a new name once everything is working.

In [ ]:
# 1. Add repository root to path to import harness
import sys
sys.path.append('../../../')  # Go up to repo root from notebooks/

# 2. Import required modules
import os
import shutil
from datetime import datetime

# 3. Import harness core functionality
from harness import ExperimentConfig, ExperimentResult, get_tracker, llm_call, run_strategy
from harness.defaults import DEFAULT_MODEL, DEFAULT_PROVIDER

# 4. Detect which providers are available on this system
providers = []

# 4a. Check if Ollama CLI is installed
if shutil.which("ollama"):
    providers.append("ollama")

# 4b. Check if MLX (Apple Silicon) is available
try:
    import mlx.core  # noqa: F401
    providers.append("mlx")
except Exception:
    pass

# 4c. Check if Anthropic API key is set
if os.getenv("ANTHROPIC_API_KEY"):
    providers.append("anthropic")

# 4d. Check if OpenAI API key is set  
if os.getenv("OPENAI_API_KEY"):
    providers.append("openai")

# 5. If no providers detected, fall back to default
if not providers:
    providers.append(DEFAULT_PROVIDER)

# 6. Print detected configuration
print("Detected providers:", providers)
print("Default model:", DEFAULT_MODEL)

# 7. Select the first available provider for testing
selected_provider = providers[0]
print("Using provider for smoke test:", selected_provider)

In [ ]:
# 1. Attempt a simple LLM call to verify the provider works
try:
    # 2. Make test call with simple prompt
    response = llm_call(
        "Say 'Hidden Layer online.'",  # Simple verification prompt
        provider=selected_provider,     # Use detected provider
        model=None                      # Use default model for provider
    )
    
    # 3. Print response details if successful
    print("Model:", response.model)
    print("Latency (s):", f"{response.latency_s:.2f}")
    print("Response:", response.text.strip())
    
# 4. If call fails, raise informative error
except Exception as exc:
    raise RuntimeError("LLM smoke test failed. Check provider configuration.") from exc

In [ ]:
# 1. Test the strategy system with a simple single-model strategy
strategy_result = run_strategy(
    "single",  # Use single-model strategy (simplest strategy)
    "List two reasons multi-agent strategies can outperform single models.",  # Test prompt
    provider=selected_provider,  # Use detected provider
    model=None,                  # Use default model
    verbose=False               # Don't print intermediate steps
)

# 2. Extract model name from result for use in tracking
selected_model = strategy_result.metadata.get("model") if strategy_result.metadata else DEFAULT_MODEL

# 3. Print strategy execution results
print("Strategy:", strategy_result.strategy_name)
print("Latency (s):", f"{strategy_result.latency_s:.2f}")
print("Output:\n", strategy_result.output.strip())

In [ ]:
# 1. Create experiment configuration for tracking
config = ExperimentConfig(
    experiment_name="quickstart_smoke",  # Identifier for this experiment
    task_type="demo",                    # Type of task
    strategy="single",                   # Strategy used
    provider=selected_provider,          # Provider used
    model=selected_model,                # Model used
)

# 2. Initialize tracker and start experiment
tracker = get_tracker()
run_dir = tracker.start_experiment(config)

# 3. Create result record with all execution details
result_record = ExperimentResult(
    config=config,
    task_input="Quickstart notebook demo task",
    output=strategy_result.output,
    latency_s=strategy_result.latency_s,
    tokens_in=strategy_result.tokens_in,
    tokens_out=strategy_result.tokens_out,
    cost_usd=strategy_result.cost_usd,
    eval_scores={},           # No evaluation scores for smoke test
    eval_metadata={},         # No evaluation metadata
    success=True,             # Test passed
    error=None,               # No errors
)

# 4. Log the result to tracker
tracker.log_result(result_record)

# 5. Finish experiment and get summary
summary = tracker.finish_experiment()

# 6. Print experiment summary if available
if summary:
    print("Logged experiment in:", run_dir)
    print("Summary runs:", summary.get("total_runs"))
    print("Average latency (s):", f"{(summary.get('avg_latency_s') or 0):.2f}")
    print("Timestamp:", datetime.fromisoformat(summary["timestamp"]))
else:
    print("No summary generated (no results logged).")